In [1]:
import sys
sys.path.append("/home/vassm/entity_alignment/kg_entity_alignment_2024/notebooks/data_prep")

In [2]:
import os
import json
from loaders import gold_folder_path, load_gold
from eval import evaluate_preds_extended, evaluate_preds

In [6]:
exactmatch_path = "/home/vassm/entity_alignment/kg_entity_alignment_2024/outputs/exact_match/label_altlabel"
exactmatch_deduplicated_path = "/home/vassm/entity_alignment/kg_entity_alignment_2024/outputs/exact_match_deduplicated"
dogtags_path = "/home/vassm/entity_alignment/kg_entity_alignment_2024/features/dogtags/version2"
top10pairs_path = "/home/vassm/entity_alignment/kg_entity_alignment_2024/features/top10pairs/bgelarge_v1"

top10pair_gpt_output = "/home/vassm/entity_alignment/kg_entity_alignment_2024/outputs/gpt4omini/top10pair_selector_v1"

In [4]:
golds = list(filter(lambda x: ".xml"in x, os.listdir(os.path.split(gold_folder_path)[0])))

In [6]:
for gold in golds:
    print(gold)
    loaded_gold = load_gold(gold)
    
    with open(os.path.join(exactmatch_path, gold.replace(".xml", ".json")), "r") as f:
        exactmatches = json.load(f)

    prec, recall, f1 = evaluate_preds(loaded_gold[0], exactmatches)
    print(prec, "#", recall, "#", f1)

marvelcinematicuniverse-marvel.xml
1.0 # 0.6766646670665867 # 0.8071556350626119
memoryalpha-memorybeta.xml
1.0 # 0.8901227976508276 # 0.9418676910908989
memoryalpha-stexpanded.xml
1.0 # 0.9168071950534008 # 0.9565982404692082
starwars-swg.xml
1.0 # 0.6253345227475469 # 0.7694840834248079
starwars-swtor.xml
1.0 # 0.8971308607417775 # 0.9457764662486167


In [7]:
golds[2:3]

['memoryalpha-stexpanded.xml']

In [8]:
for gold in golds[2:3]:
    print(gold)
    loaded_gold = load_gold(gold)

    with open(os.path.join(exactmatch_path, gold.replace(".xml", ".json")), "r") as f:
        exactmatches = json.load(f)

    g1, g2 = gold.replace(".xml", "").split("-")
    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_forward.json"), "r") as f:
        forward_top10 = json.load(f)
    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_backward.json"), "r") as f:
        backward_top10 = json.load(f)
    with open(os.path.join(exactmatch_deduplicated_path, g1 + "-" + g2 + "_leftright.json"), "r") as f:
        deduplicated_leftright = json.load(f)
    with open(os.path.join(exactmatch_deduplicated_path, g1 + "-" + g2 + "_rightleft.json"), "r") as f:
        deduplicated_rightleft = json.load(f)
    
    forward_unioned = exactmatches + [[elem[0], elem[1][0]] for elem in forward_top10]
    backward_unioned = exactmatches + [[elem[1][0], elem[0]] for elem in backward_top10]

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], exactmatches)
    print("Exact:", prec, "#", recall, "#", f1)

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], deduplicated_leftright)
    print("Deduplicated Exact left-right:", prec, "#", recall, "#", f1)

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], deduplicated_rightleft)
    print("Deduplicated Exact right-left:", prec, "#", recall, "#", f1)

    prec, recall, f1, fo_tp, fo_fn, fo_fp = evaluate_preds_extended(loaded_gold[0], forward_unioned)
    print("Forward:", prec, "#", recall, "#", f1)
    
    prec, recall, f1, ba_tp, ba_fn, ba_fp = evaluate_preds_extended(loaded_gold[0], backward_unioned)
    print("Backward:", prec, "#", recall, "#", f1)

memoryalpha-stexpanded.xml
Exact: 0.8006872852233677 # 0.9168071950534008 # 0.8548218029350105
Deduplicated Exact left-right: 0.9653731343283583 # 0.9089376053962901 # 0.9363057324840766
Deduplicated Exact right-left: 0.9642857142857143 # 0.9106239460370995 # 0.9366869037294016
Forward: 0.8003714020427113 # 0.9690837549184935 # 0.8766844647851514
Backward: 0.7995348837209302 # 0.9662731871838112 # 0.8750318147111223


In [13]:
print(len(backward_unioned))
print(len(exactmatches))

3983
3872


In [15]:
print(len(ex_tp), len(ex_fn), len(ex_fp))
print(len(fo_tp), len(fo_fn), len(fo_fp))
print(len(ba_tp), len(ba_fn), len(ba_fp))

1631 148 0
1725 54 0
1714 65 0


In [37]:
len(ex_fn)

1779

In [33]:
len(loaded_gold[0])

1779

In [34]:
print(len(exactmatches))
print(len(forward_unioned))
print(len(backward_unioned))

3872
3990
3983


In [13]:
forward_top10

[['http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/A.F.',
  ['http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Alyson_Foxton',
   0.6120147705078125]],
 ['http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Aldebaran_III',
  ['http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Aldebaran_Prime',
   0.6789420247077942]],
 ['http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Argo_(shuttlecraft)',
  ['http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Argo_type',
   0.7384515404701233]],
 ['http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Armstrong_(disambiguation)',
  ['http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Armstrong_(prototype)',
   0.5751357078552246]],
 ['http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Barstow_(Commodore)',
  ['http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Barstow',
   0.8033466339111328]],
 ['http://dbkwik.web

In [6]:
unioned_files = list(filter(lambda x: "unioned" in x, os.listdir(top10pair_gpt_output)))

In [10]:
df_rows = list()

for gold in golds:
    print(gold)
    loaded_gold = load_gold(gold)


    with open(os.path.join(exactmatch_path, gold.replace(".xml", ".json")), "r") as f:
        exactmatches = json.load(f)

    g1, g2 = gold.replace(".xml", "").split("-")

    if g1 == "marvelcinematicuniverse":
        g1 = "mcu"

    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_unioned.json"), "r") as f:
        unioned_top10 = json.load(f)
    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_forward.json"), "r") as f:
        forward_top10 = json.load(f)
    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_backward.json"), "r") as f:
        backward_top10 = json.load(f)
    with open(os.path.join(exactmatch_deduplicated_path, gold.replace(".xml", "_leftright.json")), "r") as f:
        deduplicated_leftright = json.load(f)
    with open(os.path.join(exactmatch_deduplicated_path, gold.replace(".xml", "_rightleft.json")), "r") as f:
        deduplicated_rightleft = json.load(f)

    forward_unioned = exactmatches + [[elem[0], elem[1][0]] for elem in forward_top10]
    backward_unioned = exactmatches + [[elem[1][0], elem[0]] for elem in backward_top10]
    unioned_unioned = exactmatches + [[elem[0], elem[1]] for elem in unioned_top10]

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], exactmatches)
    print("Exact:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "exactmatch", prec, recall, f1])


    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], deduplicated_leftright)
    print("Deduplicated Exact left-right:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "exactmatch", prec, recall, f1])

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], deduplicated_rightleft)
    print("Deduplicated Exact right-left:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "exactmatch", prec, recall, f1])

    prec, recall, f1, fo_tp, fo_fn, fo_fp = evaluate_preds_extended(loaded_gold[0], forward_unioned)
    print("Forward:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "Forward", prec, recall, f1])

    prec, recall, f1, ba_tp, ba_fn, ba_fp = evaluate_preds_extended(loaded_gold[0], backward_unioned)
    print("Backward:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "Backward", prec, recall, f1])

    prec, recall, f1, un_tp, un_fn, un_fp = evaluate_preds_extended(loaded_gold[0], unioned_unioned)
    print("Unioned:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "Unioned", prec, recall, f1])


marvelcinematicuniverse-marvel.xml
Exact: 0.6438356164383562 # 0.6766646670665867 # 0.6598420590816029
Deduplicated Exact left-right: 0.8839071257005604 # 0.6622675464907019 # 0.757201646090535
Deduplicated Exact right-left: 0.8847077662129704 # 0.6628674265146971 # 0.7578875171467764
Forward: 0.6241550247859396 # 0.8308338332333534 # 0.7128152341739579
Backward: 0.6522748375116063 # 0.8428314337132573 # 0.7354095786443338
Unioned: 0.6753381893860562 # 0.7786442711457708 # 0.7233212594037336
memoryalpha-memorybeta.xml
Exact: 0.8292052123744156 # 0.8901227976508276 # 0.8585848182099083
Deduplicated Exact left-right: 0.9440255883024903 # 0.8824345969033636 # 0.9121916220541972
Deduplicated Exact right-left: 0.9436893203883495 # 0.8822210357714896 # 0.9119205298013245
Forward: 0.8202380952380952 # 0.9564335290977042 # 0.8831156026620656
Backward: 0.8224144276775855 # 0.9544046983449013 # 0.8835071417980527
Unioned: 0.8374668686103749 # 0.9446876668446342 # 0.8878518741532441
memoryalpha-s

In [14]:
df_rows = list()

for gold in golds:
    print(gold)
    loaded_gold = load_gold(gold)


    with open(os.path.join(exactmatch_path, gold.replace(".xml", ".json")), "r") as f:
        exactmatches = json.load(f)

    g1, g2 = gold.replace(".xml", "").split("-")

    if g1 == "marvelcinematicuniverse":
        g1 = "mcu"

    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_unioned.json"), "r") as f:
        unioned_top10 = json.load(f)
    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_forward.json"), "r") as f:
        forward_top10 = json.load(f)
    with open(os.path.join(top10pair_gpt_output, g1 + "-" + g2 + "_top10pair_selected_backward.json"), "r") as f:
        backward_top10 = json.load(f)
    with open(os.path.join(exactmatch_deduplicated_path, gold.replace(".xml", "_leftright.json")), "r") as f:
        deduplicated_leftright = json.load(f)
    with open(os.path.join(exactmatch_deduplicated_path, gold.replace(".xml", "_rightleft.json")), "r") as f:
        deduplicated_rightleft = json.load(f)

    forward_unioned = deduplicated_rightleft + [[elem[0], elem[1][0]] for elem in forward_top10]
    backward_unioned = deduplicated_rightleft + [[elem[1][0], elem[0]] for elem in backward_top10]
    unioned_unioned = deduplicated_rightleft + [[elem[0], elem[1]] for elem in unioned_top10]

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], exactmatches)
    print("Exact:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "exactmatch", prec, recall, f1])


    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], deduplicated_leftright)
    print("Deduplicated Exact left-right:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "dedup_lr", prec, recall, f1])

    prec, recall, f1, ex_tp, ex_fn, ex_fp = evaluate_preds_extended(loaded_gold[0], deduplicated_rightleft)
    print("Deduplicated Exact right-left:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "dedup_rl", prec, recall, f1])

    prec, recall, f1, fo_tp, fo_fn, fo_fp = evaluate_preds_extended(loaded_gold[0], forward_unioned)
    print("Forward:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "Forward", prec, recall, f1])

    prec, recall, f1, ba_tp, ba_fn, ba_fp = evaluate_preds_extended(loaded_gold[0], backward_unioned)
    print("Backward:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "Backward", prec, recall, f1])

    prec, recall, f1, un_tp, un_fn, un_fp = evaluate_preds_extended(loaded_gold[0], unioned_unioned)
    print("Unioned:", prec, "#", recall, "#", f1)
    df_rows.append([gold, "Unioned", prec, recall, f1])


marvelcinematicuniverse-marvel.xml
Exact: 0.6438356164383562 # 0.6766646670665867 # 0.6598420590816029
Deduplicated Exact left-right: 0.8839071257005604 # 0.6622675464907019 # 0.757201646090535
Deduplicated Exact right-left: 0.8847077662129704 # 0.6628674265146971 # 0.7578875171467764
Forward: 0.7937062937062938 # 0.8170365926814637 # 0.8052024830032516
Backward: 0.837068443367656 # 0.8290341931613677 # 0.8330319469559976
Unioned: 0.8985200845665962 # 0.7648470305938813 # 0.8263123784834737
memoryalpha-memorybeta.xml
Exact: 0.8292052123744156 # 0.8901227976508276 # 0.8585848182099083
Deduplicated Exact left-right: 0.9440255883024903 # 0.8824345969033636 # 0.9121916220541972
Deduplicated Exact right-left: 0.9436893203883495 # 0.8822210357714896 # 0.9119205298013245
Forward: 0.9231968405736853 # 0.9485317672183663 # 0.9356928424711646
Backward: 0.9262277951933124 # 0.9465029364655633 # 0.936255611301822
Unioned: 0.94679473343406 # 0.9367859049652963 # 0.9417637271214643
memoryalpha-stexp

In [15]:
import pandas as pd

pdf = pd.DataFrame(df_rows, columns=["dataset", "Matching", "Prec", "Recall", "F1"])

In [16]:
pdf

,dataset,Matching,Prec,Recall,F1
0,marvelcinematicuniverse-marvel.xml,exactmatch,0.643836,0.676665,0.659842
1,marvelcinematicuniverse-marvel.xml,dedup_lr,0.883907,0.662268,0.757202
2,marvelcinematicuniverse-marvel.xml,dedup_rl,0.884708,0.662867,0.757888
3,marvelcinematicuniverse-marvel.xml,Forward,0.793706,0.817037,0.805202
4,marvelcinematicuniverse-marvel.xml,Backward,0.837068,0.829034,0.833032
5,marvelcinematicuniverse-marvel.xml,Unioned,0.898520,0.764847,0.826312
6,memoryalpha-memorybeta.xml,exactmatch,0.829205,0.890123,0.858585
7,memoryalpha-memorybeta.xml,dedup_lr,0.944026,0.882435,0.912192
8,memoryalpha-memorybeta.xml,dedup_rl,0.943689,0.882221,0.911921
9,memoryalpha-memorybeta.xml,Forward,0.923197,0.948532,0.935693


In [54]:
pdf.groupby(["dataset", "Matching"])

In [9]:
unioned_top10

[]

In [44]:
def get_tp(golds, preds):
    tps = list()
    remaining = list()
    for pred in preds:
        found = False
        for gold in golds:
            if gold[0] == pred[0] and gold[1] == pred[1]:
                found = True
                tps.append(pred)
                break
        if found is False:
            remaining.append(pred)
    return tps, remaining


def get_fn(golds, preds):
    fns = list()
    for gold in golds:
        found = False
        for pred in preds:
            if gold[0] == pred[0] and gold[1] == pred[1]:
                found = True
                break
        if found is False:
            fns.append(gold)
    return fns


def get_fp(golds, preds):
    fps = list()
    remaining = list()
    for pred in preds:
        found = False
        for gold in golds:
            if (gold[0] == pred[0] and gold[1] != pred[1]) or (gold[0] != pred[0] and gold[1] == pred[1]):
                found = True
                fps.append(pred)
                break
        if found is False:
            remaining.append(pred)
    return fps, remaining


def evaluate_preds(golds, preds):
    return evaluate_preds_extended(golds, preds)[:3]


def evaluate_preds_extended(golds, preds):
    fns = get_fn(golds, preds)
    tps, remaining = get_tp(golds, preds)
    fps, remaining2 = get_fp(golds, remaining)

    precision = len(tps) / (len(tps) + len(fps))
    recall = len(tps) / (len(tps) + len(fns))
    f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1, tps, fns, fps

In [ ]:
loaded_gold[0], \
    exactmatches

In [39]:
gold_elements = loaded_gold[0]
fns = get_fn(gold_elements, exactmatches)
tps, remaining = get_tp(gold_elements, exactmatches)
fps, remaining2 = get_fp(gold_elements, remaining)

In [41]:
print(len(gold_elements), len(exactmatches), len(fns), len(tps), len(remaining))

1429 3228 147 1282 1946


In [ ]:
len(fps)

In [ ]:
gold_elements = loaded_gold[0]
fns = get_fn(golds, exactmatches)
tps, remaining = get_tp(gold_elements, exactmatches)
fps, remaining = get_fp(golds, remaining)